# Tłumaczenie działania funkcji `wygladz()` z pliku `synal.py`

Funkcja musi liczyć średnią typu _running mean_, z zastrzeżeniem specjalnego zachowania na brzegach zakresu. Z komentarza dokumentacyjnego dostajemy takie warunki:

In [ ]:
"""Wygładzanie sygnału filtrem uśredniającym.
    Uwaga! Wartości na brzegach zakresu są wygładzane we fragmencie okna
    znajdującym się nad zakresem, np. dla promienia 1 (długość okna 3):
    [ 1 2 6 4 5]
    [ 1.5 ]
    [ 3 ]
    [ 4 ]
    [ 5 ]
    [ 4.5 ]
    Args:
    sygnal (numpy.array): sygnał
    promien (int): promien uśredniania (długość okna = 2 x promien + 1)
    Returns:
    numpy.array: wygładzony sygnał
    Raises:
    ValueError: jeśli podano ujemny promień
    """

## Argumenty wejściowe `sygnal` i `promien`
Wystarczy dodać je w deklaracji funkcji:

In [ ]:
def wygladz(sygnal, promien):
    pass

## Zwracanie `ValueError` w określonym momencie
Tutaj należy sprawdzić warunek i podnieść error w przypadku jego spełnienia

In [ ]:
if promien < 0:
    raise ValueError("Promień nie powinien być ujemny")

## Wyliczanie wartości
Najważniejszy element programu. W jego podstawowej wersji chcemy liczyć średnią z każdego kolejnego $n$ liczb, ale nie zawsze jest to możliwe. Rozważmy tablicę `[3, 2, 1, 0, 1, 2, 3]`. Podając promień równy $1$, na samym początku trafiamy na miejce gdzie możemy pójść o 1 w prawo, ale nie pójdziemy o 1 w lewo. Dlatego w takich miejscach musimy wziąć tyle z tego zakresu ile się da i policzyć średnią z tego.
## Jak rozdzielić te przypadki?
Zauważmy, że punkt, od którego możemy brać pełne zakresy (o szerokości 2 * `promien` + 1) ma zawsze indeks równy zmiennej `promien` (bo dopiero od takiej wartości możemy sięgnąć odpowiednio daleko na lewo). W przypadku końca tablicy, jesteśmy oddaleni od ostatniego indeksu również o promień. Jak to wygląda w kodzie?

In [ ]:
        if i < promien:
            # akcja 1
        elif i > len(sygnal) - 1 - promien:
            # akcja 2
        else:
            # akcja 3

## Dalsza budowa kodu
Musimy zwracać dane w tablicy `numpy` więc powinniśmy je do niej zapisać. Zainicjujmy ją:

In [ ]:
# pamiętajmy o zaimportowaniu numpy
import numpy as np

tab = np.array([], dtype="float_")

okno = 2 * promien + 1

Zauważmy, że `dtype` jest taki sam jak w tablicach w funkcjach testujących. Przy okazji w zmiennej `okno` umieszczamy szerokość okna średniej wyliczoną z promienia
## Jak policzyć te średnie?
Po 2h myślenia zdecydowałem, że najlepiej policzyć z pomocą sum cząstkowych tablic. Jak to działa? Sumy cząstkowe tablicy `[1, 2, 3, 4]` to `[1, 3, 6, 10]`. Każda kolejna liczba w tablicy sum to suma wszystkich liczb z pierwszej tablicy następująych do danego indeksu. Zatem w przypadku kiedy mamy za małe okienko z lewej strony, weźmiemy sumę pierwszych $n$ liczb i podzielimy przez ich ilość, w przypadku zakresu gdzie okno się mieści odejmiemy sumę z początku okna od sumy z końca okna i podzielimy przez szerokość okna, a na końcu postąpimy bardzo podobnie do początku. Zatem uzupełniamy nasz blok z instrukcjami `if`:

In [ ]:
# najpierw liczymy sumy cząstkowe
sumy_cz = np.cumsum(sygnal)

# teraz wszystko trafia do pętli for bo chcemy takie sprawdzenie dla każdej liczby w tablicy
for i in range(len(sygnal)): # ! dobra to jest tak rozjebane, że ci jupytera napiszę <- o tak zareagowałem
    if i < promien:
        tab = np.append(tab, sumy_cz[i + promien] / (i + promien + 1))
    elif i > len(sygnal) - 1 - promien:
        tab = np.append(tab, (sumy_cz[-1] - sumy_cz[i - (promien + 1)]) / (len(sygnal) - i + promien))
    else:
        tab = np.append(tab, np.average(sygnal[i - promien:(i + promien + 1)]))

Konstrukcja `append` jest trochę inna niż w samym Pythonie. Musimy podać nazwę tablicy, do której chcemy zapisać tą tablicę, do której dopisujemy. Jeśli chcemy uzyskać taki efekt jak:

In [ ]:
tab.append(x)

Musimy zrobić:

In [ ]:
tab = np.append(tab, x)

## Ostatnie modyfikacje
Teraz pozostało nam zwrócić wartość i sprawdzić czy w testach wszystko się zgadza. Zwracamy wartość:

In [ ]:
return tab

Uruchamiamy testy i mamy jeden błąd w teście, który wygląda następująco:

In [ ]:
def test_wygladz_sygnal_za_duze_okno(prosty_sygnal):
    wygladzony_sygnal = sygnal.wygladz(prosty_sygnal, 8)
    assert (wygladzony_sygnal == np.mean(prosty_sygnal)).any()

Oznacza to, że musimy się przygotować na sytuację kiedy ktoś wpisze promień tworzący za duże okno na cały zakres. Mamy wtedy zwrócić średnią wszystkich wartości w tablicy:

In [ ]:
    if okno > len(sygnal) and len(sygnal) > 0: # funkcja np.mean nie działa kiedy sygnał jest pusty, stąd drugi warunek
        return np.mean(sygnal)

Teraz mamy już wszystko, nasz kod zdaje każdy test i wygląda następująco:

In [ ]:
import numpy as np

def wygladz(sygnal, promien):
    if promien < 0:
        raise ValueError("Promień nie powinien być ujemny")
    tab = np.array([], dtype="float_")
    okno = 2 * promien + 1
    if okno > len(sygnal) and len(sygnal) > 0:
        return np.mean(sygnal)
    sumy_cz = np.cumsum(sygnal)
    for i in range(len(sygnal)):
        if i < promien:
            tab = np.append(tab, sumy_cz[i + promien] / (i + promien + 1))
        elif i > len(sygnal) - 1 - promien:
            tab = np.append(tab, (sumy_cz[-1] - sumy_cz[i - (promien + 1)]) / (len(sygnal) - i + promien))
        else:
            tab = np.append(tab, np.average(sygnal[i - promien:(i + promien + 1)]))
    return tab

Jest 4:30, nie dam rady opisać tak drugiej funkcji bo jest dużo bardziej skurwiała.

[Link do całego kodu tego zadania i zadania 1](https://github.com/mathuntley/trash-can)